In [1]:
import os
import numpy as np
import pandas as pd
import time
import random
import PIL.Image as Image
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.metrics import roc_curve, auc , f1_score, accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None

In [7]:
train_dir = '/kaggle/input/final-label/FINAL'
csv_file='/kaggle/input/wsi-csv/selected_data.csv'

#val_lib = ''
val_dir = '/kaggle/input/label-val/LABEL_CHECKING'

project='train_convergence'             
output = os.path.join('/kaggle/working')

batch_size = 1
nepochs = 5

test_every = 1           # related to validation set, if needed
weights = 0.5            # weight of a positive class if imbalanced

lr = 1e-4                # learning rate
weight_decay = 1e-4      # l2 regularzation weight
 
best_auc_v = 0           # related to validation set, if needed

In [3]:
#dataloader
def Image_filter(folder_path,dimensions ,threshold=3000):
    print("DIMENSIONS: ",dimensions)
    filtered_images = []
    if dimensions:
        filtered_images.append(dimensions[0])
        prev_height, prev_width = dimensions[0][:2]
        #print("First Image : ",prev_height,"  ",prev_width)
        
    for height, width, img_file in dimensions[1:]:
        #print("Height : Width : ",height,"  ",width," ",img_file)
        #print("Filtered Images : ",filtered_images)
        if abs(height - prev_height) <= threshold:
            filtered_images.append((height, width, img_file))
            i = 0
            while i < len(filtered_images) - 1:
                if abs(filtered_images[i][0] - filtered_images[i + 1][0]) > threshold:
                    res = filtered_images.pop(i)
                    print("Popped : ", res)
                else:
                    i += 1
        else:
            if filtered_images and abs(height - filtered_images[-1][0]) < threshold:
                filtered_images.append((height, width, img_file))
            if filtered_images[-1][0] > height:
                res = filtered_images.pop()
                filtered_images.append((height, width, img_file))
                print("Pop : ", res)
                
        if not filtered_images:
                filtered_images.append((height, width, img_file))

        prev_height, prev_width = height, width

    return filtered_images

def get_image_dimensions(folder_path):
    image_files = os.listdir(folder_path)
    dimensions = []

    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)
        try:
            img = cv2.imread(img_path)

            if img is None:
                print(f"Error reading image {img_path}")
                continue

            height, width = img.shape[:2]
            dimensions.append((height, width, img_file))
        except Exception as e:
            print(f"Exception occurred while reading image {img_path}: {e}")
            continue

    return dimensions

def zero_pad_and_convert_to_tensor(folder_path, image_names, max_height, max_width):
    padded_images = []

    for img_name in image_names:
        img_path = os.path.join(folder_path, img_name)
        try:
            img = cv2.imread(img_path)
        
            if img is None:
                print(f"Error reading image {img_path}")
                continue
        
            height, width = img.shape[:2]
            pad_height = max_height - height
            pad_width = max_width - width

            padded_img = np.pad(img, ((0, pad_height), (0, pad_width), (0, 0)), mode='constant', constant_values=0)
            padded_images.append(padded_img)
        except Exception as e:
            print(f"Exception occurred while processing image {img_path}: {e}")
            continue

    tensors = [torch.tensor(img, dtype=torch.float32).permute(2, 0, 1) for img in padded_images]
    return tensors

class MPdataset(data.Dataset):
    def __init__(self, path_dir=None, csv_file=None, transform=None, mult=2, is_validation=False):
        self.folder_path = path_dir
        if not is_validation:
            image_files = os.listdir(self.folder_path)
            dimensions = []

            for img_file in image_files:
                img_path = os.path.join(self.folder_path, img_file)
                try:
                    img = cv2.imread(img_path)

                    if img is None:
                        print(f"Error reading image {img_path}")
                        continue

                    height, width = img.shape[:2]
                    dimensions.append((height, width, img_file))
                    #print("Dimesions: ",dimensions)
                except Exception as e:
                    print(f"Exception occurred while reading image {img_path}: {e}")
                    continue

            dimensions.sort(reverse=True, key=lambda x: x[0])
            
            filtered_images = Image_filter(self.folder_path,dimensions)
            #print("FILTERED IMAGES:")
        else:
            filtered_images = get_image_dimensions(self.folder_path)

        if not filtered_images:
            print("No images left after filtering.")
        else:
            for height, width, filename in filtered_images:
                print(f"- {filename} ({height}x{width})")
            print("****************************")
                
        max_height = max(dim[0] for dim in filtered_images)
        max_width = max(dim[1] for dim in filtered_images)

        filtered_image_names = [img[2] for img in filtered_images]
        self.tensors = zero_pad_and_convert_to_tensor(self.folder_path, filtered_image_names, max_height, max_width)
        self.image_names = filtered_image_names
        
        # Read CSV file and match names to get labels
        if csv_file:
            labels_df = pd.read_csv(csv_file)
            labels_dict = {row['PATIENT'][:12]: 1 if row['isMSIH'] == 'MSIH' else 0 for idx, row in labels_df.iterrows()}
            self.targets = [labels_dict.get(img_name[:12], 0) for img_name in self.image_names]
            print("SELF TARGETS : ",self.targets)
        else:
            self.targets = [0] * len(self.tensors)  # Default to zero if no CSV provided

        self.transform = transform
        self.mult = mult
        self.mode = None

    def setmode(self, mode):
        self.mode = mode

    def maketraindata(self, idxs):
        self.t_data = [(self.image_names[x], self.tensors[x], self.targets[x]) for x in idxs]

    def shuffletraindata(self):
        self.t_data = random.sample(self.t_data, len(self.t_data))

    def __getitem__(self, index):
        if self.mode == 1:
            img = self.tensors[index]
            target = self.targets[index]
            if self.transform is not None:
                img = self.transform(img)
            return img, target
        elif self.mode == 2:
            img_name, img, target = self.t_data[index]
            if self.transform is not None:
                img = self.transform(img)
            return img, target

    def __len__(self):
        if self.mode == 1:
            return len(self.tensors)
        elif self.mode == 2:
            return len(self.t_data)


In [4]:
def calc_roc_auc(target, prediction):
    fpr, tpr, thresholds = roc_curve(target, prediction)
    roc_auc = auc(fpr, tpr)
    return roc_auc

def calculate_accuracy(output, target):
    #print("Output: ",output)
    #print("Targets : ",target)
    # Since output is a probability, threshold at 0.5 to get class predictions
    preds = (output > 0.5).float().cpu()
    #print("preds: ",preds)
    target = target.cpu()
    acc = accuracy_score(target.view_as(preds), preds)
    return acc


# also add F1 score
def calculate_f1(output, target):
    # Convert probabilities to binary predictions using a threshold of 0.5
    preds = (output > 0.5).float().cpu()
    target = target.cpu()
    
    # Calculate F1 score
    f1 = f1_score(target.view_as(preds), preds)
    return f1


#function to calculate mean of data grouped per slide, used for aggregating tile scores into slide score
def group_avg(groups, data):
    order = np.lexsort((data, groups))
    groups = groups[order]
    data = data[order]
    unames, idx, counts = np.unique(groups, return_inverse=True, return_counts=True)
    group_sum = np.bincount(idx, weights=data)
    group_average = group_sum / counts
    return group_average

#function to find index of max value in data grouped per slide
def group_max(groups, data, nmax):
    out = np.empty(nmax)
    out[:] = np.nan
    order = np.lexsort((data, groups))
    groups = groups[order]
    data = data[order]
    index = np.empty(len(groups), 'bool')
    index[-1] = True
    index[:-1] = groups[1:] != groups[:-1]
    out[groups[index]] = data[index]
    return out

def Pk_score(all_labels, all_probs):
    if isinstance(all_labels, list):
        all_labels = np.array(all_labels)
    if isinstance(all_probs, list):
        all_probs = np.array(all_probs)
        
    all_probs = np.ravel(all_probs) 
    
    # Print shapes and data to debug
    #print("all_probs:", all_probs)
    #print("all_probs shape:", all_probs.shape)
    #print("all_labels:", all_labels)
    #print("all_labels shape:", all_labels.shape)

    # Ensure that all_labels and all_probs have the same length
    if len(all_labels) != len(all_probs):
        raise ValueError("Length of all_labels and all_probs must be the same")
    
    # Check if all_labels contains both classes
    if np.unique(all_labels).size < 2:
        raise ValueError("all_labels must contain both positive and negative samples")

    # Calculate precision-recall curve
    precision, recall, _ = precision_recall_curve(all_labels, all_probs)
    #print("Precision:", precision)
    #print("Recall:", recall)

    # Calculate average precision score
    try:
        auc_pk_score = average_precision_score(all_labels, all_probs)
        if np.isnan(auc_pk_score):
            raise ValueError("Calculated AUC Pk-Score is NaN")
    except ValueError as e:
        print(f"Error calculating average precision score: {e}")
        auc_pk_score = None
    
    print("AUC Pk-Score:", auc_pk_score)
    
    return auc_pk_score


In [5]:
#baseline cnn model to fine tune
model = models.resnet18(True)
model.fc = nn.Linear(model.fc.in_features, 1)
model.cuda()

if weights==0.5:
    criterion = nn.CrossEntropyLoss().cuda()
else:
    w = torch.Tensor([1-weights,weights])
    criterion = nn.CrossEntropyLoss(w).cuda()
    
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=lr)

cudnn.benchmark = True

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
model = models.resnet18(True)
model.fc = nn.Linear(model.fc.in_features, 1)
model.cuda()

if weights == 0.5:
    criterion = nn.BCEWithLogitsLoss().cuda()
else:
    w = torch.Tensor([1 - weights, weights])
    criterion = nn.CrossEntropyLoss(w).cuda()

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=lr)

cudnn.benchmark = True

normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.1, 0.1, 0.1])

trans = transforms.Compose([
    normalize,
])

trans_Valid = transforms.Compose([
    normalize,
])

train_dset = MPdataset(path_dir=train_dir, csv_file=csv_file, transform=trans, is_validation=False)
train_loader = torch.utils.data.DataLoader(
    train_dset,
    batch_size=batch_size, shuffle=False,
    num_workers=4, pin_memory=False
)

if val_dir:
    val_dset = MPdataset(path_dir=val_dir, csv_file=csv_file, transform=trans_Valid, is_validation=True)
    val_loader = torch.utils.data.DataLoader(
        val_dset,
        batch_size=batch_size, shuffle=False,
        num_workers=4, pin_memory=False
    )

# Open output files for convergence metrics
output_dir = '/kaggle/working/'
os.makedirs(output_dir, exist_ok=True)

with open(os.path.join(output_dir, 'train_convergence.csv'), 'w') as fconv:
    fconv.write('epoch,loss,accuracy,f1,Roc_auc,average_precision_score\n')

with open(os.path.join(output_dir, 'valid_convergence.csv'), 'w') as fconv:
    fconv.write('epoch,accuracy,Roc_auc,f1,average_precision_score\n')

train_dset.setmode(1)
# Prepare all tiles of training set slides
num_tiles = len(train_dset)  # Assuming len(train_dset) gives the number of tiles
print("TILES Selected : ",num_tiles)

train_dset.maketraindata(np.arange(num_tiles))

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DIMENSIONS:  [(4119, 3003, 'TCGA-A6-5657_nonMSIH6.png'), (3163, 3195, 'TCGA-A6-2681_nonMSIH3.png'), (3074, 2690, 'TCGA-A6-5661_MSIH7.png')]
- TCGA-A6-5657_nonMSIH6.png (4119x3003)
- TCGA-A6-2681_nonMSIH3.png (3163x3195)
- TCGA-A6-5661_MSIH7.png (3074x2690)
****************************
SELF TARGETS :  [0, 0, 1]
- TCGA-A6-5665_MSIH8.png (4044x2479)
- TCGA-A6-5666_nonMSIH9.png (5232x4554)
****************************
SELF TARGETS :  [1, 0]
TILES Selected :  3


In [9]:
def train(run, loader, model, criterion, optimizer):
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    all_outputs = []
    all_targets = []

    for i, (input, target) in enumerate(loader):
        input = input.cuda()
        target = target.cuda().float().view(-1, 1)  # Ensure target is of the correct type and shape
        
        # Debugging: Print shapes of inputs and targets
        #print(f"Input shape: {input.shape}, Target shape: {target.shape}")
        
        # Forward pass
        output = model(input)
        
        # Calculate loss
        #print("OUTPUTTTTTTTTT: ",output)
        #print("TARGETTTTTTTTT: ",target)
        
        loss = criterion(output, target)
        
        # Debugging: Print loss value
        #print(f"Loss: {loss.item()}")
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy
        running_loss += loss.item() * input.size(0)
        
        # Convert output to probabilities
        probs = torch.sigmoid(output)
        
        # Predictions based on threshold of 0.5
        preds = (probs > 0.5).float()
        
        # Calculate accuracy
        acc = calculate_accuracy(preds, target)
        running_acc += acc * input.size(0)
        
        all_outputs.append(probs)
        all_targets.append(target)

        if i % 100 == 0:
            # Compute metrics on the current batch
            batch_f1 = calculate_f1(preds, target)
            batch_auc_roc = calc_roc_auc(target.cpu().numpy(), probs.detach().cpu().numpy())
            
            # Print epoch training loss, accuracy, F1, and ROC-AUC for the current batch
            print('Training\tEpoch: [{:3d}/{:3d}]\tBatch: [{:3d}/{}]\tLoss: {:0.4f}\tAccuracy: {:0.2f}%\tF1: {:0.2f}\tROC-AUC: {:0.2f}'
                .format(run + 1, nepochs, i + 1, len(loader), 
                        running_loss / ((i + 1) * input.size(0)), 
                        (100 * running_acc) / ((i + 1) * input.size(0)), 
                        batch_f1, batch_auc_roc))

    # Compute metrics for the entire epoch
    all_outputs = torch.cat(all_outputs)
    all_targets = torch.cat(all_targets)
    
    # For the full dataset, calculate metrics using concatenated outputs and targets
    epoch_f1 = calculate_f1((all_outputs > 0.5).float(), all_targets)
    epoch_auc_roc = calc_roc_auc(all_targets.cpu().numpy(), all_outputs.detach().cpu().numpy())
    epoch_acc = (100 * running_acc) / len(loader.dataset)
    epoch_pk_score = Pk_score(all_targets.cpu().numpy(), all_outputs.detach().cpu().numpy())

    return running_loss / len(loader.dataset), epoch_acc, epoch_f1, epoch_auc_roc, epoch_pk_score



import torch.nn.functional as F

def inference(run, loader, model):
    model.eval()
    running_acc = 0.0
    all_outputs = []
    all_targets = []
    
    batch_size = loader.batch_size

    probs = torch.FloatTensor(len(loader.dataset))
    preds = torch.FloatTensor(len(loader.dataset))
    
    with torch.no_grad():
        for i, (input, target) in enumerate(loader):
            input = input.cuda()
            target = target.cuda()
            
            logits = model(input)
            y = torch.sigmoid(logits).view(-1)  # Flatten y

            # Get binary predictions
            pr = (y > 0.5).float()            
            #print("Y : ",y)
            #print("Pr : ",pr)
            # Ensure the size matches
            preds[i * batch_size:i * batch_size + input.size(0)] = pr.cpu().detach()
            probs[i * batch_size:i * batch_size + input.size(0)] = y.cpu().detach()
            #print("PROBSSSSSSSSSSS : ",probs)
            
            # Debug output
            #print("Labels:", target.cpu().numpy())
            #print("Probabilities:", y.cpu().numpy())
            
            acc = calculate_accuracy(pr, target)
            running_acc += acc * input.size(0)
            
            all_outputs.append(logits)
            all_targets.append(target)

            if i % 100 == 0:
                batch_f1 = calculate_f1(pr, target)
                batch_auc_roc = calc_roc_auc(target.cpu().numpy(), y.cpu().numpy())
                #batch_pk_score = Pk_score(target.cpu().numpy(), pr.cpu().numpy())
                
                print('Inference\tRun: [{:3d}]\tBatch: [{:3d}/{}]\tAccuracy: {:0.2f}%\tF1: {:0.2f}\tROC-AUC: {:0.2f}'
                      .format(run + 1, i + 1, len(loader), 
                              acc, batch_f1, batch_auc_roc))

    all_outputs = torch.cat(all_outputs)
    all_targets = torch.cat(all_targets)
    
    overall_acc = (100 * running_acc) / len(loader.dataset)
    overall_f1 = calculate_f1(preds, all_targets)
    overall_auc_roc = calc_roc_auc(all_targets.cpu().numpy(), probs.cpu().numpy())
    overall_pk_score = Pk_score(all_targets.cpu().numpy(), probs.cpu().numpy())
    
    print('Inference\tRun: [{:3d}]\tOverall\tAccuracy: {:0.2f}%\tF1: {:0.2f}\tROC-AUC: {:0.2f}\tPk-Score: {:0.2f}'
          .format(run + 1, overall_acc, overall_f1, overall_auc_roc, overall_pk_score))

    return probs.cpu().numpy(), overall_acc, preds.cpu().numpy(), overall_f1, overall_auc_roc, overall_pk_score


In [10]:
# Measure data loading time
start_time = time.time()

# Initialize best AUC value
best_auc_v = 0.0

# Loop through epochs
for epoch in range(nepochs):
    train_dset.shuffletraindata()
    train_dset.setmode(2)
    
    # Train the model and get the metrics
    loss, acc, f1, auc_roc, auc_pk_score = train(epoch, train_loader, model, criterion, optimizer)
    
    # Measure elapsed time so far
    print("--- {:0.2f} minutes ---".format((time.time() - start_time)/60.))
    
    # Print training metrics
    print('Training\tEpoch: [{}/{}]\tLoss: {:0.4f}\tAccuracy: {:0.4f}\tF1: {:0.4f}\tROC-AUC: {:0.4f}\tPk-Score: {:0.4f}'
          .format(epoch + 1, nepochs, loss, acc, f1, auc_roc, auc_pk_score))
    
    # Log training metrics to CSV
    with open(os.path.join(output, 'train_convergence.csv'), 'a') as fconv:
        fconv.write('{},{:0.4f},{:0.4f},{:0.4f},{:0.4f},{:0.4f}\n'.format(epoch + 1, loss, acc, f1, auc_roc, auc_pk_score))
    
    # Validation if needed
    if val_dir and (epoch+1) % test_every == 0:
        val_dset.setmode(1)
        val_probs, val_acc, val_preds, val_f1, val_auc_roc,val_pk_score  = inference(epoch, val_loader, model)
        
        # Compute additional metrics using calculate_auc_and_f1
        auc_pk_score = Pk_score(val_dset.targets, val_probs)

        # Update best AUC
        best_auc_v = max(best_auc_v, val_auc_roc)
        
        # Print validation metrics including PR-AUC
        print('Validation\tEpoch: [{}/{}]\tval_acc: {:0.4f}\tROC-AUC: {:0.4f}\tPR-AUC: {:0.4f}\tbest so far: {:0.4f}'
              .format(epoch + 1, nepochs, val_acc, val_auc_roc, val_pk_score, best_auc_v))
        
        # Log validation metrics to CSV
        with open(os.path.join(output, 'valid_convergence.csv'), 'a') as fconv:
            fconv.write('{},{:0.4f},{:0.4f},{:0.4f},{:0.4f}\n'.format(epoch+1, val_acc, val_auc_roc,val_f1,val_pk_score))
        
        # Save best model
        if val_auc_roc > best_auc_v:
            best_auc_v = val_auc_roc
            obj = {
                'epoch': epoch+1,
                'state_dict': model.state_dict(),
                'best_auc_v': best_auc_v,
                'optimizer': optimizer.state_dict()
            }
            torch.save(obj, os.path.join(output, 'checkpoint_best.pth'))
    
    # Measure accumulated elapsed time so far
    print("--- {:0.2f} minutes ---".format((time.time() - start_time)/60.))


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


Training	Epoch: [  1/  5]	Batch: [  1/3]	Loss: 0.8848	Accuracy: 0.00%	F1: 0.00	ROC-AUC: nan
AUC Pk-Score: 0.3333333333333333
--- 1.13 minutes ---
Training	Epoch: [1/5]	Loss: 0.6881	Accuracy: 66.6667	F1: 0.0000	ROC-AUC: 0.0000	Pk-Score: 0.3333


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


Inference	Run: [  1]	Batch: [  1/2]	Accuracy: 0.00%	F1: 0.00	ROC-AUC: nan
AUC Pk-Score: 1.0
Inference	Run: [  1]	Overall	Accuracy: 50.00%	F1: 0.00	ROC-AUC: 1.00	Pk-Score: 1.00
AUC Pk-Score: 1.0
Validation	Epoch: [1/5]	val_acc: 50.0000	ROC-AUC: 1.0000	PR-AUC: 1.0000	best so far: 1.0000
--- 1.24 minutes ---


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


...

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


Training	Epoch: [  4/  5]	Batch: [  1/3]	Loss: 0.6646	Accuracy: 100.00%	F1: 1.00	ROC-AUC: nan
AUC Pk-Score: 1.0
--- 1.48 minutes ---
Training	Epoch: [4/5]	Loss: 0.4450	Accuracy: 100.0000	F1: 1.0000	ROC-AUC: 1.0000	Pk-Score: 1.0000


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


Inference	Run: [  4]	Batch: [  1/2]	Accuracy: 0.00%	F1: 0.00	ROC-AUC: nan
AUC Pk-Score: 1.0
Inference	Run: [  4]	Overall	Accuracy: 50.00%	F1: 0.00	ROC-AUC: 1.00	Pk-Score: 1.00
AUC Pk-Score: 1.0
Validation	Epoch: [4/5]	val_acc: 50.0000	ROC-AUC: 1.0000	PR-AUC: 1.0000	best so far: 1.0000
--- 1.51 minutes ---


KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Define paths
output_dir = '/kaggle/working/'
train_csv_path = os.path.join(output_dir, 'train_convergence.csv')
valid_csv_path = os.path.join(output_dir, 'valid_convergence.csv')

# Read CSV files
train_df = pd.read_csv(train_csv_path)
valid_df = pd.read_csv(valid_csv_path)

# Plot training metrics
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_df['epoch'], train_df['loss'], label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_df['epoch'], train_df['accuracy'], label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

# Plot validation metrics
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(valid_df['epoch'], valid_df['Acc'], label='Accuracy ')
plt.xlabel('Epoch')
plt.ylabel('acc')
plt.title('Accuracy over Epochs')
plt.legend()


plt.tight_layout()
plt.show()
